# Power Line Detection Algorithm: Step by Step Explanation

## 1. Image Loading and Preprocessing

```python
img = cv2.imread(image_path)
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) if len(img.shape) == 3 else img
gray = cv2.equalizeHist(gray)
```

- **Histogram Equalization**: This redistributes pixel intensities to enhance contrast. Mathematically, it transforms the histogram to be approximately uniform. For each pixel value i, the new value becomes:
  ```
  equalized(i) = floor((CDF(i) - CDFmin) × (L-1) / (M×N - CDFmin))
  ```
  Where CDF is the cumulative distribution function, M×N is image size, and L is the number of gray levels (256). This is from the slides ahahsah

## 2. Edge Detection with Sobel Operator

```python
sobel_y = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=3)
abs_sobel_y = np.absolute(sobel_y)
sobel_y_8u = np.uint8(255 * abs_sobel_y / np.max(abs_sobel_y))
```

- **Sobel Operator** is a discrete differentiation operator that computes an approximation of the gradient of the image intensity. In this case, we're detecting horizontal edges by applying the y-direction kernel.

- The Sobel y-direction kernel for a 3×3 filter is:
  ```
  [ 1  2  1]
  [ 0  0  0]
  [-1 -2 -1]
  ```

- For each pixel at position (x,y), the filter response is calculated as:
  ```
  G_y(x,y) = ∑∑ K(i,j) * I(x+i-1, y+j-1)
  ```
  Where K is the kernel and I is the image ALSO from the slides ahahahahahruanrjnrgfd

- **Normalization**: The `abs_sobel_y / np.max(abs_sobel_y)` normalizes values to [0,1], then multiplying by 255 scales to standard 8-bit range [0,255].

## 3. Region of Interest Selection

```python
h, w = gray.shape
margin = int(0.30 * w)
x_start = margin
x_end = w - margin
cropped_edges = sobel_y_8u[:, x_start:x_end]
```

- This creates a region of interest (ROI) by taking the middle 40% of the image width (margins of 30% on each side).
- In image coordinates, this is selecting a submatrix from the original matrix.

## 4. Thresholding

```python
binary = cv2.threshold(cropped_edges, 50, 255, cv2.THRESH_BINARY)[1]
```

- **Binary Thresholding**: For each pixel value p at position (x,y):
  ```
  binary(x,y) = 255 if p > 50 else 0
  ```
- This creates a binary image where edge pixels are white (255) and non-edge pixels are black (0).

## 5. Column-by-Column Pixel Analysis

```python
for x in range(binary.shape[1]):
    edge_pixels = np.where(binary[:, x] > 0)[0]
```

- For each column in the image, we find all positions where edge pixels exist.
- `np.where()` returns indices where the condition is true, giving us y-coordinates of all edge pixels in this column.

## 6. Pixel Grouping

```python
pixel_groups = []
current_group = [edge_pixels[0]]
for i in range(1, len(edge_pixels)):
    if edge_pixels[i] - edge_pixels[i-1] < 10:
        current_group.append(edge_pixels[i])
    else:
        pixel_groups.append(current_group)
        current_group = [edge_pixels[i]]
```

- This groups vertically contiguous edge pixels by checking their proximity.
- If two consecutive pixels are less than 10 pixels apart vertically, they belong to the same group.
- This clustering step helps identify distinct line segments in each column.

## 7. Group Position Calculation

```python
group_avg_y = [sum(group) / len(group) for group in pixel_groups]
```

- For each group of pixels, we calculate the mean y-position. This represents the center of each detected line segment in the current column.
- Mathematically, this is the arithmetic mean: μ = (∑y) / n where n is the number of pixels in the group.

## 8. Phase Assignment and Tracking

```python
# First initialization
if phase_a_avg_y is None and len(sorted_groups) >= 1:
    phase_a_avg_y = sorted_avg_y[0]
    phase_a_points.append((x + x_start, int(sorted_avg_y[0])))
```

and

```python
# Subsequent matching
distances = []
if phase_a_avg_y is not None and 'A' not in assigned_phases:
    distances.append(abs(avg_y - phase_a_avg_y))
    
# Finding closest phase
min_idx = np.argmin(distances)
```

- **Initial Phase Assignment**: When first encountering edge groups, we assign phases top-to-bottom (A-B-C).

- **Ongoing Tracking**: For subsequent columns, we match edge groups to phases based on proximity to the running average position of each phase.

- **Distance Calculation**: We compute the absolute difference between each detected group's position and the running average position of each phase: d = |y_group - y_phase_avg|

- **Minimum Distance Assignment**: We assign the group to the phase with the smallest distance, provided it's within our tolerance threshold.

## 9. Running Average Updates

```python
phase_a_avg_y = 0.7 * phase_a_avg_y + 0.3 * avg_y
```

- This implements an **Exponentially Weighted Moving Average (EWMA)** for phase tracking.
- It's essentially a low-pass filter that smooths out position estimates.
- The formula is: new_avg = α * old_avg + (1-α) * new_value, where α = 0.7 controls the weight of history.
- This gives more weight (70%) to historical positions and less weight (30%) to the new observation, providing smoothness and continuity while still adapting to gradual changes.

## 10. Drawing and Visualization

```python
# Draw points
cv2.circle(result_img, point, 2, color, -1)

# Connect points
cv2.line(result_img, points[i-1], points[i], color, 2)
```

- This visualizes detected points and connects them to show continuous power lines.
- Points from the same phase are connected sequentially to form polylines.

## Mathematical Insights

1. **Edge Detection**: The Sobel operator is essentially a finite difference approximation of the image gradient, highlighting areas of rapid intensity change.

2. **Clustering**: The grouping algorithm implements a simple form of one-dimensional density-based clustering with a fixed distance threshold.

3. **Phase Tracking**: The algorithm uses a combination of:
   - Nearest neighbor matching (minimum distance assignment)
   - Exponential smoothing (weighted running average)
   - Threshold-based association (y_tolerance parameter)

4. **Coordinate Spaces**: Note that we're working in multiple coordinate spaces:
   - Original image coordinates
   - Cropped ROI coordinates (shifted by x_start)
   - Column-specific coordinates within the ROI

5. **Robustness Elements**:
   - The weighted running average prevents jumps from noise
   - The tolerance threshold (y_tolerance) prevents misassignments
   - Grouping nearby pixels reduces impact of noise
   - The algorithm can handle missing phases in some columns
